# Wrangling Lake Tahoe Area Open Street Map Data

## A Data Wrangling Project by Eric Gordon

Openstreetmap.org is an openly publiclly sourced interactive map, where users enter any data they see fit, and all data is freely availible to download and analyze. This report takes a 123.8 MB XML file of data dowloaded from http://www.openstreetmap.org/export#map=10/39.0981/-119.9220 of the greater Lake Tahoe area, and audits the quality of data that has been entered for this area.  Also, the data will be converted from XML format to CSV files, and then stored in a database to dive deeper into the the data to find some other insights. Finally this project suggests further recomendations to investigating this data. If you want to see more info on the data format of this XML data file, please feel free to check out the websites wikipedia at http://wiki.openstreetmap.org/wiki/OSM_XML. 

# Data Auditing

Open street map data relies on users inputing "Nodes" and "Ways" to makle up the data for their maps, meaning that it was worth first looking at some of the downloaded data before importing this data into an SQL database. While running code to audit the data, some of the following stats were found about the map. 

#### Type of XML Tags In Lake Tahoe Area Map Data:

{'node': 593474, 'member': 10197, 'nd': 650393, 'tag': 176948, 'bounds': 1, 'note': 1, 'meta': 1, 'osm': 1, 'way': 35616, 'relation': 351}

That means in this file for the Lake Tahoe map, there were about 593,474 nodes and 35,616 ways. Nodes are items marked on the map that are classified on a map as having a singular points, like a building resturaunt, or or natural landmark. Ways on the other hand, are map objects that require boundaries like parks, boarders, or roadways. Both objects, Nodes and Ways can have one or more 'Tags' which are data objects that give ways and nodes their attributes. We will look further at these tags later, this info is just worth noting now before moving on with this analysis.

#### Issues with Data
While auding this data in more detail with python, there were more problems that were found even before importing the data into a SQL database. 

1. First, while auditing the tags, many tags had the key value of "addr:street" and these tags were thus audited to see what values useres inputted for street names. The values of these tags had some inconsistentcies due to what must be indvidiauls entering the street labels differently. For example a couple of highways were mislabeled like "Hwy 80 PM 27.4," where the PM 27.4 does not make sense in context of a highway label. Additionally different tags used different abbrivations for the same endings. "River Ranch Road" and "River Ranch Rd" were entered in different tags, where they obviously refer to the same road. Thus these street values would be cleaned before importing into a SQL database. 
2. Athoer issue was found when auditing the tags with the key 'phone' label. These tags were attributes that refered to phone numbers for either businesses or stores, however formats for inputed phone numbers were highly inconssistent. Here are a few examples of the phone numbers found in this map data for four different entries:
        "+1 530-541-5160," "530-541-5227," "1 775-315-9574," and "1-866-795-2709."

      Thus all data inputted as "NV" and "CA" would be converted to "Nevada" and "California" respectivelClearly these phone numbers all have different formats, specifically with regards to how useres inputed the start of the phone numbers and the country "+1" code. This will therefore also be cleaned before being entered into our database, to create a slightly more uniform set of data.
3. The key "addr:state" was used with tags to locate which state items in this map were found. The Lake Tahoe area is interesting because the lake is right on the border of the California and Nevada state line. Therefore there were tags for items in both California and Nevada. Therefore while audding the user inputed tags for state values, the following four ways of identifiyg state were found:
              set(['CA', 'Nevada', 'California', 'NV'])

   Before going any further then, it was clear that all data inputted as "NV" and "CA" would be converted to "Nevada" and "California" respectively before inputting as an SQL File. But it was good to see that not other states were found in this data. 
4. Finally, the major problem with this data seemed to quite simply be that a large lack of data for the area seems to be missing. While auditing for phone numbers, only 83 unique phone numbers were printed having the tag key of "Phone." This seems very low for the area this data comes from, especially considering that South Lake Tahoe alone has a population of roughly 21,000 people, and this city only reprsents a small corner of the map shown above. Further investigation will be done once the data is in SQL searchable form, but for now it is worth noting that the data seemed very incomplete for the map area.

### Cross-Field Validation
One last validation that was conducted before fixing some of the data issues, was a cross-field validation between city names and states. That is, because Lake Tahoe is encompassed in both California and Nevada, it is important to check to make sure each element that was tagged with a "addr:city" key and a "addr:state" key, was valid with the correct cities in each state. Thus a cross-field check was run with the following output:

'Nevada': set(['Carson City', 'Zephyr Cove', 'Stateline', 'Incline Village', 'Reno', 'Gardnerville', 'Elk Point', 'Minden']),

'California': set(['Truckee', 'Olympic Valley', 'South Lake Tahoe', 'Markleeville'])}

All of the output cities are in fact in the correct states. While auditing, if a node or way had both a "addr: state" tag and a "addr: city" tag, the state and city were consistent. This means that no users associated a city with the wrong state, which is a good thing! However it is worth noting that 

### Fixing Data Issues
Thus with these considerations understood, we cleaned the phone number data and the minor issues with the street endings programatically while importing all of the data into CVS files. 
After these files were transfered to CVS, we compiled them all within a single database so that further analysis could be completed within this writeup using SQL querries. The funtions below show how the phone numbers, states and street names were cleaned during this exporting process, with an example of each being cleaned:

In [1]:
import re
def update_name(name, mapping):
    street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
    better_name=name
    m=street_type_re.search(name)
    if m:
        if m.group() in mapping.keys():
            better_street_type = mapping[m.group()]
            better_name = street_type_re.sub(better_street_type, name)
    return better_name

def update_number(number):
    phone_start= re.compile('^\s*?\+?1\-?\s?')
    better_number=number
    if re.search(phone_start,number):
            better_number = re.sub(phone_start,'',number)
    return better_number

def update_state(state):
    new_state=state
    state_mapping={"CA":"California", "NV":"Nevada", "California":"California",\
                   "Nevada":"Nevada"}
    if state in state_mapping.keys():
        new_state=state_mapping[state]
    return new_state

print "Phone Example"
print "+1 530-544-4050"
print "Changes to " + update_number('+1 530-544-4050')
print''
print "Street Example"
print"Market St"
print "Changes to " + update_name("Market St",{"St":"Street"})
print''
print "State Example"
print"NV"
print "NV changes to " + update_state("NV")

Phone Example
+1 530-544-4050
Changes to 530-544-4050

Street Example
Market St
Changes to Market Street

State Example
NV
NV changes to Nevada


*NOTE: These again are just examples of the functions used. The full code of fixing the data and importing the data to CVS files in a seaperate full programming file. 

# SQL Queries and Data Investigation

Now with our data for Lake Tahoe able to be evaluated, lets look at some of the data we have. First, let's complete an SQL query to see what types of tags are most common. Remember, Tags are the data elements that give Nodes and Ways their attributes in this online map, such as a name, or the type of element we have. The "key" value for each tag will tell us what type of tag we have for each datapoint, and these keys are fairly standardized, so they will help us figure out what type of data we have.

## Queries 1 and 2

In [3]:
import sqlite3
import csv

sqlite_file='lake_tahoe.db'
conn=sqlite3.connect(sqlite_file)
cur=conn.cursor()
cur.execute('''
SELECT key, count(*) FROM nodes_tags group by key order by count(*) desc limit 15;
            ''')
output=cur.fetchall()
print "Top Nodes Tags"
print(output)
cur.execute('''
SELECT key, count(*) FROM ways_tags group by key order by count(*) desc limit 15;
            ''')
print ''
print "Top Ways Tags"
output=cur.fetchall()
print(output)

Top Nodes Tags
[(u'power', 3197), (u'highway', 2331), (u'name', 1154), (u'source', 1097), (u'natural', 791), (u'ele', 570), (u'amenity', 491), (u'feature_id', 411), (u'created', 400), (u'county_id', 355), (u'state_id', 355), (u'housenumber', 300), (u'attribution', 202), (u'place', 153), (u'import_uuid', 141)]

Top Ways Tags
[(u'highway', 18561), (u'name', 12275), (u'source', 11773), (u'cfcc', 9775), (u'county', 9762), (u'reviewed', 9412), (u'building', 8321), (u'name_base', 7192), (u'name_type', 6724), (u'tlid', 5175), (u'zip_left', 4857), (u'separated', 4623), (u'zip_right', 4351), (u'surface', 4341), (u'landuse', 3004)]


Thus from this first queries, we see that a lot of the data we have is taged with being a highway element, or the name of something. Additionally, there were 3,197 nodes that are taged as having something to do with 'power' which is interesting, because I would not expect an open sourced map to contain thousands of user ented data points taged with 'power'. We will need to look into these keys more to see some details, but first its worth looking at all of the highway values in both the ways tags and the nodes tags at the same time. The values search will give us more info on what these map elements actually are.

### 'Highway' Tags, Query 3

In [4]:
cur.execute('''
SELECT value, count(*) FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) as tags WHERE key='highway' group by value order by count(*) desc;
            ''')
output=cur.fetchall()
print(output)

[(u'residential', 8418), (u'service', 2833), (u'track', 2024), (u'path', 1694), (u'turning_circle', 1317), (u'footway', 1287), (u'tertiary', 506), (u'crossing', 444), (u'secondary', 407), (u'unclassified', 314), (u'stop', 212), (u'traffic_signals', 198), (u'cycleway', 187), (u'construction', 174), (u'motorway', 164), (u'motorway_link', 131), (u'trunk', 128), (u'primary', 124), (u'bus_stop', 85), (u'proposed', 74), (u'motorway_junction', 49), (u'secondary_link', 27), (u'trunk_link', 23), (u'living_street', 19), (u'steps', 13), (u'priority', 8), (u'street_lamp', 8), (u'pedestrian', 5), (u'mini_roundabout', 4), (u'primary_link', 4), (u'turning_loop', 3), (u'road', 2), (u'tertiary_link', 2), (u'checkpoint', 1), (u'give_way', 1), (u'trailhead', 1), (u'yes', 1)]


Clearly these user entered tags that have been labeled "highway", are majoily users identifying the type of roads on the open street map, with some other roadway elements also being included in this category. This is also cool because we can see that the majority of roads in the Lake Tahoe area are either residential or service roads, which could verly likely be very different than what you may find in a major city. Also it seems that temporary or future roads may also be included in this open street map data from values in this list as "proposed" and "construction." It is helpful seeing what the highway data shows.

### 'Power' Tags, Query 4
Now let's investigate and take a look back at what the 'power' tags include. We will have to see what things are being classified as 'power.'

In [5]:
cur.execute('''
SELECT value,count() FROM nodes_tags WHERE key='power' group by value order by count() desc;
''')
output=cur.fetchall()
print(output)

[(u'tower', 1765), (u'pole', 1432)]


Now we can see that the nodes and ways that have been labeled in the 'power' category are actually simply refering to either electrical towers or poles. This is not surprising in now understanding what these things are, but is shocking when you remember that all of this map data is user generated, so some user or users inputed all of these towers and poles. This is slightly surprising. It is also shocking that the most common node tag was 'power', instead of other things like buildings or resturaunts, or even 'natural' elements in such a beautiful part of the world as Lake Tahoe. 

### 'Name' Tags, Queries 5 and 6
Let's now investigate what is classified as a 'Name." 

In [6]:
cur.execute('''
SELECT value, count(*) FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) as tags WHERE key='name' group by value order by count(*) desc limit 20;
            ''')
output=cur.fetchall()
print(output)

cur.execute('''
SELECT value, count(*) FROM nodes_tags WHERE key='name' group by value order by count(*) desc limit 20;
            ''')
output=cur.fetchall()
print ""
print "Names other than Roads"
print(output)

[(u'BP', 81), (u'Alan S. Hart Freeway', 68), (u'Tahoe Rim Trail', 67), (u'North-South Freeway', 53), (u'North Lake Boulevard', 48), (u'hanger', 48), (u'Carson City Freeway', 40), (u'CA 89', 39), (u'Wedge Parkway', 39), (u'Donner Pass Road', 38), (u'Mount Watson Road', 33), (u'Pacific Crest Trail (TRT)', 31), (u'Carson-Reno Highway', 29), (u'Mount Rose Highway', 27), (u'Carson Valley Freeway', 26), (u'South Virginia Street', 26), (u'West College Parkway', 23), (u'Northridge Drive', 22), (u'Truckee Bike Path', 21), (u'Fairview Drive', 19)]

Names other than Roads
[(u'Starbucks', 9), (u'Subway', 6), (u'7-Eleven', 3), (u'Bank of America', 3), (u'Dollar Tree', 3), (u'KFC', 3), (u'Port of Subs', 3), (u'Round Table Pizza', 3), (u'Sand Harbor', 3), (u'Taco Bell', 3), (u'Alpine Meadows', 2), (u'Angora Peak', 2), (u'Barker Pass', 2), (u'Basecamp Hotel', 2), (u'Blue Dog Gourmet Pizza', 2), (u'Blue Jay Lodge', 2), (u'CVS', 2), (u'Carnelian Bay', 2), (u'Carson Ranger District', 2), (u'Casino Fandan

The two queries above showed that the majority of 'names' refer to street names or other driving landmarks. The second query therefore was used to take aways all of the road names (coming from ways tags) and evaluate what the "names" of nodes were. The second query shows that a lot of the names are for chains of resturaunts like fast food joints, or names of other places around Lake Tahoe. This makes it clear also because there were 1,154 Node tags labeled as "names," but that the most common Node name was starbucks, with only 9 entries. This means that there are a wide variety of unique Node names inputed in this map region.

# Summary Stats, Final Queries 

The querries below are results to give us some final additional stats before a conclusion.

In [7]:
import pprint
cur.execute('''
SELECT COUNT (*) FROM nodes''')
output=cur.fetchall()
print "Number of Nodes"
print(output)
print ''
cur.execute('''
SELECT COUNT (*) FROM ways ''')
output=cur.fetchall()
print "Number of Ways"
pprint.pprint(output)
print ''
cur.execute('''
SELECT COUNT(DISTINCT uid)FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways);''')
output=cur.fetchall()
print "Number of Distinct Users Who Entered Data"
print(output)
cur.execute('''
SELECT user, count(*) FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways)
group by user order by count(*) desc limit 20;        ''')
output=cur.fetchall()
print ''
print "Top 20 Contributing Users"
pprint.pprint(output)

Number of Nodes
[(593474,)]

Number of Ways
[(35616,)]

Number of Distinct Users Who Entered Data
[(462,)]

Top 20 Contributing Users
[(u'theangrytomato', 96429),
 (u'nmixter', 78206),
 (u'AndrewBuck', 60608),
 (u'woodpeck_fixbot', 49360),
 (u'Sebastien R', 27382),
 (u'MelanieOriet', 25941),
 (u'paulmach', 19365),
 (u'lucaswoj', 17300),
 (u'abschiff', 15388),
 (u'Marshy8', 14523),
 (u'wallclimber21', 9945),
 (u'Apo42', 9330),
 (u'RJones1917', 8943),
 (u'Eureka gold', 8861),
 (u'Spanholz', 8692),
 (u'KindredCoda', 7862),
 (u'dchiles', 7161),
 (u'drbrady', 7015),
 (u'Hiker Dan', 6102),
 (u'JHPrides', 5016)]


The Data above shows some final stats about our Lake Tahoe Data, including the number of Nodes and Ways in our database, (which match our numbers from the start of this investigation) and the top contributors to the open map data. It is worth noting that the users with tens of thousands of entries seems too many to be mannually entered. Which may suggest that some of the data has been entered programatically. 

# Suggestions for Future Data Imporovements

The Lake Tahoe area data we analyized, which is characterised as being very full of nature and a small human population, seemed unique in several ways. It seems very interesting that the most frequent node as a key called "power", with also Highways and roads being the only other items tagged very commonly. 

My suggestion for future analysis of this area is for an attempt to audit for completeness. While this is a highly subjective feeling, there seems to be a lot of data missing, even in the most raw form. Again noting that only 83 unique phone numbers were found, which makes me wonder how many additional Nodes could be incoorporated in this region of the open soruced map. Also while investigating some of the names of nodes, only three names were '7-Eleven', but I know for a fact that there are that many 7-Elevens just in North Lake Tahoe, meaning there are other 7-Elevens missing from this map data. If there was some sort of publically availible business database, or phone directory database, it would be very interesting to evaluate and analyze the completeness of this open sourced map for the Lake Tahoe Area with other databases that already exist. Also it would be insightful to compare a rural area's percent completeness with that of a different map in an urban setting. The distinction between a rural area and an urban area could possibly be insightful.

The obvious difficulties of auditing for completeness in any map, especially an opened sourced map, is setting the standard of what would be considered "missing." A lot of the Lake Tahoe Area map seemed to also revolve around natural elements like bays and peaks , so it may be hard to find a database or source to compare these natural elements of the goegraphy, especially in data form, in order to audit for completeness. Maybe a phonebook directory or registered business databases could handle the for completeness business, but again, having reliable and the correct sources of data to compare this map data to maybe the overall most difficult aspect of auditing for completeness. Overall the Data for this area was pretty well entered with few errors or non-sensicle data entries, but further investigation on its completeness would in my opnion be the most beneficial future analysis. 

# Resources

XML File for this Analysis from:
http://wiki.openstreetmap.org/wiki/OSM_XML

XML File Format Help:
http://wiki.openstreetmap.org/wiki/OSM_XML

Help with Converting a CVS file to SQL Database:
http://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python

Overall General Help:
https://www.udacity.com/course/data-wrangling-with-mongodb--ud032,
https://discussions.udacity.com/c/nd002-p3-data-wrangling/p3-wrangle-openstreetmaps-data-with-sql